# Visualizations of the modified loss landscape $\mathbf{f_\lambda}$

This notebook is used for generating plots of how the (linearized) piecewise constant loss $f$ changes with hyperparameter $\lambda$. The method is proposed in [Differentiation of Blackbox Combinatorial Solvers](https://arxiv.org/abs/1912.02175). 

Implemented are solvers for **GraphMatching**, **MultiGraphMatching**, **TravellingSalesman**, **Ranking**, and **ShortestPath.**

TODO: Write the formula for f_lambda

# Test iPyVolume plotting

Run to test whether ipyvolume installation succeeded (and all required jupyter extentions are enabled)

In [ ]:
import numpy as np
import ipyvolume as ipv

V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0
ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)

# Loss landscape plots

Some example problems are generated in the following:

The magic constants are selected so that we obtain interesting sections of the high-dimensional function.

WARNING: It's 2D sections of high-D functions, sometimes may not be intuitive (interpolation happens in different dimension)

In [ ]:
from solvers_to_visualize import RankingBBSolver, TSPBBSolver, GraphMatchingBBSolver,  \
                                 ShortestPathBBSolver, MultiGraphMatchingBBSolver

In [ ]:
lambdas = [1e-6, 1, 10, 20, 40, 100, 200]
# Each plot will be a sweep over these values of lambda

### Ranking

In [ ]:
rk_solv = RankingBBSolver(sequence_length=10, seed=41, wn_f=30, ws_f=20, y_s=1)  # the last three arguments change the values of w and y_grad
rk_solv.plot_flambda(lambdas=lambdas, partitions=80, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0))

### TSP

In [ ]:
tsp_solv = TSPBBSolver(num_nodes=5, seed=42, wn_f=10, ws_f=20, y_f=1)
tsp_solv.plot_flambda(lambdas=lambdas, partitions=10, bounds1=(-1.0, 1.0), bounds2=(-1.0, 1.0), show_axes=False, show_box=False)

### Graph Matching

In [ ]:
gm_solv = GraphMatchingBBSolver(num_nodes_l=[5, 5], num_edges_l=[6, 6], seed=42, wn_f=10, y_f=0.1)
gm_solv.plot_flambda(lambdas=lambdas, partitions=40, bounds1=(-1.0, 1.0), bounds2=(-1.0, 1.0))

### Shortest path

In [ ]:
sp_solv = ShortestPathBBSolver(num_nodes=10, seed=410, y_f=0.01)
sp_solv.plot_flambda(lambdas=lambdas, partitions=30, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0))

# Use of margin

TODO: We recommend to use the solvers with a margin as done in bla and bla.
TODO: explain margin


Specifying multiple margins will plot the landscape for all lambdas, for each margin. The slider controls both lambda and margin.
In the example below, we use shortest path and examine interesting behaviour around the origin. This happens, because for $w_1 = w_2 = 0$ with the provided $w$-cut (which has the shift set to zero) we get to the point were the higher-dimensional true $w$ reaches the point 0. This is a special point as for strictly positive weights, we always have a minimum of the linearized loss ($f = w \cdot \frac{df}{dy} = 0 \cdot \frac{df}{dy} = 0$) and around this point all possible values of the loss are achieved. An introduced margin can deal with this by shifting the loss landscape away from the 0-point to the actual minimum.

In [ ]:
sp_solv = ShortestPathBBSolver(num_nodes=10, seed=410, y_f=0.01, ws_f=0)
sp_solv.plot_flambda(lambdas=lambdas, margins=[0, 0.5], partitions=30, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0), plot_cost=True)

Finally, the multigraph solver takes a while:

In [ ]:
mgm_solv = MultiGraphMatchingBBSolver(num_nodes_l=[5, 5], num_edges_l=[6, 6], seed=42, wn_f=10, y_f=0.1)
mgm_solv.plot_flambda(lambdas=lambdas, bounds1=(-1.0,1.0), bounds2=(-1.0,1.0), partitions=10)

In [ ]:
#mgm_solv = MultiGraphMatchingBBSolver(num_nodes_l=[3, 3, 3], num_edges_l=[6, 6, 6], seed=42, wn_f=10, y_f=0.1)
#mgm_solv.plot_flambda(lambdas=[0.0000001, 1, 10], bounds1=(-1.0,1.0), bounds2=(-1.0,1.0), partitions=10)

# Customize! Run your own solvers and cuts!

Problems can be added by specifying the solver and the input-generation function as shown below.
The input-generation should provide:
- the config for the solver
- a two-dimensional cut of a higher dimensional $w$-landscape 
- some gradient $\frac{dL}{dy}$ of the loss with respect to $y$, for which the perturbed linearized loss landscape $f_\lambda$ should be plotted. 

For details on the provided helper functions to generate these inputs, as well as the calculations done for the plots, see utils.flamba_utils.

The challenge is always to find values for the $w$-cut and the $\frac{dL}{dy}$ that produce interesting results, as in producing multiple different outputs of the solver. To find these regions there are various parameters that can be changed, including shifts and factors for changing the randomized inputs to values that "fit" the solver, as well as the seed. The boundaries for the plotting region can also be changed.